In [1]:
require 'nn'
require 'hdf5'

### Loading the data:

In [76]:
myFile = hdf5.open('6-grams.hdf5','r')
data = myFile:all()
myFile:close()

In [92]:
data

{
  valid_output : LongTensor - size: 3370x50
  train_1000 : LongTensor - size: 696825x7
  test : LongTensor - size: 3761x55
  nwords : LongTensor - size: 1
  train : LongTensor - size: 772670x7
  valid : LongTensor - size: 3370x55
}


### Creating the training set:

We need to remove the counts (used for the Count based language model)

In [81]:
nwords = 10001
train = data['train']:narrow(2,1,6)

In [82]:
train_input = train:narrow(2,1,5)
train_output = train:narrow(2,6,1)

In [89]:
N = 6
dwin = N-1
hid1 = 30.
hid2 = 100
nnlm = nn.Sequential()

tanh = nn.Sequential()
tanh:add(nn.LookupTable(nwords,hid1/dwin))
tanh:add(nn.View(1,-1,hid1))
tanh:add(nn.Squeeze()) -- this layer is to go from a 1xAxB tensor to AxB dimensional tensor (https://groups.google.com/forum/#!topic/torch7/u4OEc0GB74k)
tanh:add(nn.Linear(hid1,hid2))
tanh:add(nn.Tanh())

nnlm:add(tanh)
nnlm:add(nn.Linear(hid2, nwords))
nnlm:add(nn.LogSoftMax())

In [84]:
nnlm:forward(train_input:narrow(1,1,1)):size()

 10001
[torch.LongStorage of size 1]



In [90]:
criterion = nn.ClassNLLCriterion()

In [86]:
dataset={};
for i=1,train:size(1) do 
  dataset[i] = {train_input[i]:view(1,5), train_output[i]}
end
function dataset:size() return train:size(1) end -- 100 examples

In [91]:
--with regularisation:

sq_mat = torch.zeros(10001,6)
norm_mat = torch.zeros(10001,6)

for i = 1, 20 do
    
    timer = torch.Timer()
    if i ~= 1 then
        sq_mat:copy(torch.pow(nnlm:findModules('nn.LookupTable')[1].weight,2))
        norm_mat:copy(torch.expandAs(sq_mat:sum(2), sq_mat))
        nnlm:findModules('nn.LookupTable')[1].weight:cdiv(norm_mat)
    end
    
    trainer = nn.StochasticGradient(nnlm, criterion)
    trainer.learningRate = 0.01
    trainer.maxIteration = 1
    trainer:train(dataset)
    print('Epoch '..i..': '..timer:time().real)
    
end


# StochasticGradient: training	


# current error = 6.4245431634655	
# StochasticGradient: you have reached the maximum number of iterations	
# training error = 6.4245431634655	
Epoch 1: 1867.2106969357	


# StochasticGradient: training	


# current error = 5.9158270506484	
# StochasticGradient: you have reached the maximum number of iterations	
# training error = 5.9158270506484	


Epoch 2: 1851.8256881237	


...

# current error = 5.1854116802198	
# StochasticGradient: you have reached the maximum number of iterations	
# training error = 5.1854116802198	
Epoch 18: 1804.7857210636	


# StochasticGradient: training	


# current error = 5.1802477525104	
# StochasticGradient: you have reached the maximum number of iterations	
# training error = 5.1802477525104	
Epoch 19: 1804.3937339783	


# StochasticGradient: training	


# current error = 5.163963303637	
# StochasticGradient: you have reached the maximum number of iterations	
# training error = 5.163963303637	
Epoch 20: 1804.8741080761	


In [137]:
--save
torch.save('nnlm1',nnlm)

In [75]:
-- Without regularisation:

trainer = nn.StochasticGradient(nnlm, criterion)
trainer.learningRate = 0.01
trainer.maxIteration = 5
trainer:train(dataset)

# StochasticGradient: training	


# current error = 4.3886357784844	


# current error = 4.1514940863395	


# current error = 4.0717284582465	


# current error = 4.0314867838883	


# current error = 4.00779912249	
# StochasticGradient: you have reached the maximum number of iterations	
# training error = 4.00779912249	


### Tentative de Validation:

In [212]:
--load:
nnlm = torch.load('../../nnlm1')

In [244]:
kag = data['valid_output']

In [246]:
valid = data['valid']
valid_topredict = valid:narrow(2,1,50)
valid_input = valid:narrow(2,51,5)

In [248]:
preds_valid = nnlm:forward(valid_input)

In [249]:
kag_pred_valid = torch.Tensor(preds_valid:size(1),50)
for i = 1,preds_valid:size(1) do
    kag_pred_valid[i]:copy(preds_valid[i]:index(1,valid_topredict[i])):exp()
end

In [250]:
norm_mat_ = torch.zeros(preds_valid:size(1),50)
norm_mat_:copy(torch.expandAs(kag_pred_valid:sum(2), kag_pred_valid))
kag_pred_valid:cdiv(norm_mat_)

In [251]:
acc = 0
for i = 1,preds_valid:size(1) do
    m,a = kag_pred_valid[i]:max(1)
    mm,aa = kag[i]:max(1)
    if aa[1] == a[1] then
        acc = acc + 1
    end
end
print(acc/3370.)

0.42581602373887	


In [138]:
function perplexity(distribution)
    local perp = torch.DoubleTensor(distribution:size(1), distribution:size(2))
    perp:copy(distribution)
    n = distribution:size(1) * distribution:size(2)
    return math.exp(-(1/n)* perp:log():sum())
end

In [252]:
perp = 0
for i = 1,preds_valid:size(1) do
    mm,aa = kag[i]:max(1)
    perp = perp + math.log(kag_pred_valid[i][aa[1]])
end
perp = math.exp(-perp/preds_valid:size(1))
print(perp)

26.309662260245	


In [253]:
kag_pred_valid[preds_valid:size(1)][aa[1]]

0.49076318656613	


In [156]:
dataset[1]

{
  1 : LongTensor - size: 1x5
  2 : LongTensor - size: 1
}


### Deep:

In [206]:
N = 6
dwin = N-1
hid1 = 30.
hid2 = 100
dnnlm = nn.Sequential()

LT = nn.Sequential()
LT:add(nn.LookupTable(nwords,hid1/dwin))
LT:add(nn.View(1,-1,hid1))
LT:add(nn.Squeeze()) 

dnnlm:add(LT)

concat = nn.ConcatTable()

lin_tanh = nn.Sequential()
lin_tanh:add(nn.Linear(hid1,hid2))
lin_tanh:add(nn.Tanh())

id = nn.Identity()

concat:add(lin_tanh)
concat:add(id)

dnnlm:add(concat)
dnnlm:add(nn.JoinTable(1))
dnnlm:add(nn.Linear(hid2+hid1, nwords))
dnnlm:add(nn.LogSoftMax())

In [207]:
t = train_input[8000]

In [208]:
tt = dnnlm:forward(t)

In [189]:
nn.JoinTable(1):forward(ttt):size()

 130
[torch.LongStorage of size 1]



In [ ]:
criterion = nn.ClassNLLCriterion()

In [272]:
sq_mat:sum(2):max()

38.705445706632	


In [214]:
--with regularisation:

sq_mat = torch.zeros(10001,6)
norm_mat = torch.zeros(10001,6)


for i = 2, 15 do
    
    timer = torch.Timer()
    if i ~= 1 then
        sq_mat:copy(torch.pow(dnnlm:findModules('nn.LookupTable')[1].weight,2))
        norm_mat:copy(torch.expandAs(sq_mat:sum(2), sq_mat))
        nnlm:findModules('nn.LookupTable')[1].weight:cdiv(norm_mat)
    end
    
    trainer = nn.StochasticGradient(dnnlm, criterion)
    trainer.learningRate = 0.01
    trainer.maxIteration = 1
    trainer:train(dataset)
    print('Epoch '..i..': '..timer:time().real)
    
    
end

# StochasticGradient: training	


# current error = 6.0753078968396	
# StochasticGradient: you have reached the maximum number of iterations	
# training error = 6.0753078968396	
Epoch 2: 2378.5694739819	


# StochasticGradient: training	


# current error = 5.9075009795102	
# StochasticGradient: you have reached the maximum number of iterations	
# training error = 5.9075009795102	
Epoch 3: 2383.4352579117	


# StochasticGradient: training	


...

# current error = 5.4319642996491	
# StochasticGradient: you have reached the maximum number of iterations	
# training error = 5.4319642996491	
Epoch 13: 4948.130671978	


# StochasticGradient: training	


# current error = 5.4142337359248	
# StochasticGradient: you have reached the maximum number of iterations	
# training error = 5.4142337359248	
Epoch 14: 2365.0248579979	


# StochasticGradient: training	


# current error = 5.3984644474863	
# StochasticGradient: you have reached the maximum number of iterations	
# training error = 5.3984644474863	
Epoch 15: 2369.2235422134	


In [254]:
kag_pred_valid = torch.Tensor(preds_valid:size(1),50)
for i = 1,preds_valid:size(1) do
    kag_pred_valid[i]:copy(dnnlm:forward(valid_input[i]):index(1,valid_topredict[i])):exp()
end

In [255]:
norm_mat_ = torch.zeros(preds_valid:size(1),50)
norm_mat_:copy(torch.expandAs(kag_pred_valid:sum(2), kag_pred_valid))
kag_pred_valid:cdiv(norm_mat_)

In [228]:
acc = 0
for i = 1,preds_valid:size(1) do
    m,a = kag_pred_valid[i]:max(1)
    mm,aa = kag[i]:max(1)
    if aa[1] == a[1] then
        acc = acc + 1
    end
end
print(acc/3370.)

0.57833827893175	


In [258]:
CE = 0
for i = 1,preds_valid:size(1) do
    mm,aa = kag[i]:max(1)
    CE = CE + math.log(kag_pred_valid[i][aa[1]])
end

In [259]:
print('Perplexity is equal to: '.. math.exp(-CE/preds_valid:size(1)))

Perplexity is equal to: 7.3734290599113	


### Testing on test data:

In [238]:
test_data = data['test']
test_topredict = test_data:narrow(2,1,50)
test_input = test_data:narrow(2,51,5)


In [241]:
kag_pred_test = torch.Tensor(test_data:size(1),50)
for i = 1,test_data:size(1) do
    kag_pred_test[i]:copy(dnnlm:forward(test_input[i]):index(1,test_topredict[i])):exp()
end

In [242]:
norm_mat_ = torch.zeros(test_data:size(1),50)
norm_mat_:copy(torch.expandAs(kag_pred_test:sum(2), kag_pred_test))
kag_pred_test:cdiv(norm_mat_)

In [243]:
filename = 'tocsv_1.f5'
myFile = hdf5.open(filename, 'w')
myFile:write(filename, kag_pred_test)
myFile:close()  

### Trying with 4-grams and changing embeddings size:

In [276]:
myFile = hdf5.open('5-grams.hdf5','r')
data5 = myFile:all()
myFile:close()

In [277]:
data5

{
  valid_output : LongTensor - size: 3370x50
  train_1000 : LongTensor - size: 620208x6
  test : LongTensor - size: 3761x54
  nwords : LongTensor - size: 1
  train : LongTensor - size: 754037x6
  valid : LongTensor - size: 3370x54
}


In [278]:
nwords = 10001
train5 = data5['train']:narrow(2,1,5)
train_input5 = train5:narrow(2,1,4)
train_output5 = train5:narrow(2,5,1)

In [279]:
dataset5={};
for i=1,train5:size(1) do 
  dataset5[i] = {train_input5[i]:view(1,4), train_output5[i]}
end
function dataset5:size() return train5:size(1) end -- 100 examples

In [280]:
kag5 = data5['valid_output']
valid5 = data5['valid']
valid_topredict5 = valid5:narrow(2,1,50)
valid_input5 = valid5:narrow(2,51,4)

In [281]:
N = 5
dwin = N-1
hid1 = 30.
hid2 = 100
dnnlm2 = nn.Sequential()

LT2 = nn.Sequential()
LT2:add(nn.LookupTable(nwords,hid1))
LT2:add(nn.View(1,-1,hid1*dwin))
LT2:add(nn.Squeeze()) 

dnnlm2:add(LT2)

concat2 = nn.ConcatTable()

lin_tanh2 = nn.Sequential()
lin_tanh2:add(nn.Linear(hid1*dwin,hid2))
lin_tanh2:add(nn.Tanh())

id2 = nn.Identity()

concat2:add(lin_tanh2)
concat2:add(id2)

dnnlm2:add(concat2)
dnnlm2:add(nn.JoinTable(1))
dnnlm2:add(nn.Linear(hid2+hid1*dwin, nwords))
dnnlm2:add(nn.LogSoftMax())

In [282]:
criterion = nn.ClassNLLCriterion()

In [285]:
kag_pred_valid5 = torch.Tensor(valid_input5:size(1),50)
norm_mat_5 = torch.zeros(valid_input5:size(1),50)
sq_mat = torch.zeros(10001,30)

val_res = torch.Tensor(10)

In [286]:
--with regularisation:

for i = 1, 10 do
    
    timer = torch.Timer()
    if i ~= 1 then
        sq_mat:zero()
        sq_mat:copy(torch.pow(dnnlm2:findModules('nn.LookupTable')[1].weight,2))
        dnnlm2:findModules('nn.LookupTable')[1].weight:div(sq_mat:sum(2):max())
    end
    
    trainer2 = nn.StochasticGradient(dnnlm2, criterion)
    trainer2.learningRate = 0.005
    trainer2.maxIteration = 1
    trainer2:train(dataset5)
    print('Epoch '..i..': '..timer:time().real)
    
    kag_pred_valid5:zero()

    for i = 1,valid5:size(1) do
        kag_pred_valid5[i]:copy(dnnlm2:forward(valid_input5[i]):index(1,valid_topredict5[i])):exp()
    end
    
    norm_mat_5:zero()
    norm_mat_5:copy(torch.expandAs(kag_pred_valid5:sum(2), kag_pred_valid5))
    kag_pred_valid5:cdiv(norm_mat_5)
    
    CE = 0
    for i = 1,kag_pred_valid5:size(1) do
        mm,aa = kag5[i]:max(1)
        CE = CE + math.log(kag_pred_valid5[i][aa[1]])
    end
    
    val_res[i] = math.exp(-CE/kag_pred_valid5:size(1))
    print('Perplexity on valid: '..val_res[i])
end

# StochasticGradient: training	


# current error = 6.4204610677058	
# StochasticGradient: you have reached the maximum number of iterations	
# training error = 6.4204610677058	
Epoch 1: 3663.0253610611	


Perplexity on valid: 6.6291954810161	


# StochasticGradient: training	
